In [4]:
import codecs
import networkx as nx
import os
import pandas as pd
import pickle
from os import listdir
from os.path import isfile, join
from networkx.drawing.nx_pydot import write_dot
from utils import printProgressBar

# column number Demonette
graph_1 = 3
graph_2 = 6
cat_1 = 8
cat_2 = 10
cstr_1 = 14
cstr_2 = 17
complexite = 19
orientation = 21
fichier_origine = 43

# column number Glawinette
lemma1 = 0
lemma2 = 1
cat1 = 2
cat2 = 3
origine_morpho = 4
origine_def = 5
BAP1 = 6
BAP2 = 7
BAPsize = 8
FAP1 = 9
FAP2 = 10
FAPsize = 11
radical = 12
FAPtype = 13

In [7]:
folders_needed = ['DG-families', 'DG-graph', 'raw_csv_files']
for f in folders_needed:
    if not os.path.exists(f):
        os.makedirs(f)